In [ ]:
import os
import glob
from osgeo import osr,gdal
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install ipython-autotime
%load_ext autotime

time: 1.21 ms (started: 2021-07-22 14:17:35 +00:00)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 1min 17s (started: 2021-07-22 14:17:35 +00:00)


In [ ]:
photon_df = pd.read_csv("/content/drive/MyDrive/PS-DATA/Photon_cloud/final.csv", index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 12.7 s (started: 2021-07-22 14:18:53 +00:00)


In [ ]:
photon_df.head()

,latitude,longitude,photon height
0,30.687413,79.451328,3601.8672
2,30.687426,79.451327,3617.4138
3,30.687439,79.451325,3610.9563
4,30.687445,79.451324,3610.0974
5,30.687464,79.451322,3615.3230


time: 20.3 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
ds = gdal.Open("/content/drive/MyDrive/PS-DATA/Badrinath_SRTMGL1Ellip.tif")
transform = ds.GetGeoTransform()
band = ds.GetRasterBand(1)
array = band.ReadAsArray()
# plt.imshow(array)
# array

time: 595 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
width = ds.RasterXSize
height = ds.RasterYSize
print(width)
print(height)

1196
771
time: 3.77 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
gt = ds.GetGeoTransform()

time: 1.23 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
origin_x = gt[0]
origin_y = gt[3]
pixel_res_x = gt[1]
pixel_res_y = gt[5]

time: 2.17 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
def latitude(n):
    return origin_y + (n/width * pixel_res_y)
def longitude(n):
    return origin_x + (n%width * pixel_res_x)

time: 2.36 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
array = np.array(array)
array = array.flatten()
dem_df = pd.DataFrame(data = array,columns = ['dem height'])
dem_df['index1']=dem_df.index
dem_df['latitude'] = dem_df.index1.apply(latitude)
dem_df['longitude'] = dem_df.index1.apply(longitude)
dem_df = dem_df.drop(['index1'],axis=1)
dem_df = dem_df[['latitude','longitude','dem height']]

time: 808 ms (started: 2021-07-22 14:19:06 +00:00)


In [ ]:
dem_df

,latitude,longitude,dem height
0,30.880694,79.206806,5513.889648
1,30.880694,79.207083,5518.893066
2,30.880694,79.207361,5483.896484
3,30.880694,79.207639,5457.899902
4,30.880694,79.207917,5441.903809
...,...,...,...
922111,30.666529,79.537639,3450.636230
922112,30.666529,79.537917,3434.635986
922113,30.666528,79.538194,3408.635986
922114,30.666528,79.538472,3390.635986


time: 19.6 ms (started: 2021-07-22 14:19:07 +00:00)


In [ ]:
def nearest_latitude_point(n):
  return int((n-origin_y)/pixel_res_y)

def nearest_longitude_point(n):
  return int((n-origin_x)/pixel_res_x)

def nearest_latitude_pointincres(n):
  return int((n-origin_y)*3/pixel_res_y)

def nearest_longitude_pointincres(n):
  return int((n-origin_x)*3/pixel_res_x)

time: 5.15 ms (started: 2021-07-22 14:19:07 +00:00)


In [ ]:
photon_df['latitude_index']= photon_df.latitude.apply(nearest_latitude_point)
photon_df['longitude_index']= photon_df.longitude.apply(nearest_longitude_point)
photon_df['latitude_index2']= photon_df.latitude.apply(nearest_latitude_pointincres)
photon_df['longitude_index2']= photon_df.longitude.apply(nearest_longitude_pointincres)

time: 25.1 s (started: 2021-07-22 14:19:07 +00:00)


In [ ]:
photon_df['container_box_id']=(photon_df['latitude_index'])*(width)+photon_df['longitude_index']

time: 115 ms (started: 2021-07-22 14:19:32 +00:00)


In [ ]:
photon_df['container_box_id2']=(photon_df['latitude_index2'])*(width)+photon_df['longitude_index2']
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id,container_box_id2
0,30.687413,79.451328,3601.8672,695,880,2087,2640,832100,2498692
2,30.687426,79.451327,3617.4138,695,880,2087,2640,832100,2498692
3,30.687439,79.451325,3610.9563,695,880,2087,2640,832100,2498692
4,30.687445,79.451324,3610.0974,695,880,2087,2640,832100,2498692
5,30.687464,79.451322,3615.3230,695,880,2086,2640,832100,2497496
...,...,...,...,...,...,...,...,...,...
304475,30.687428,79.168337,4218.9990,695,-138,2087,-415,831082,2495637
304476,30.687428,79.168337,4218.0440,695,-138,2087,-415,831082,2495637
304477,30.687429,79.168338,4215.5747,695,-138,2087,-415,831082,2495637
304478,30.687428,79.168337,4218.1950,695,-138,2087,-415,831082,2495637


time: 480 ms (started: 2021-07-22 14:19:32 +00:00)


In [ ]:
photon_df=photon_df.groupby(['container_box_id2']).mean()
photon_df

,latitude,longitude,photon height,latitude_index,longitude_index,latitude_index2,longitude_index2,container_box_id
container_box_id2,,,,,,,,
220963,30.863485,79.179274,5934.822300,61.0,-99.0,185.0,-297.0,72857.0
221058,30.863491,79.188049,5329.479165,61.0,-67.0,185.0,-202.0,72889.0
221234,30.863479,79.204328,5572.857538,61.0,-8.0,185.0,-26.0,72948.0
221243,30.863501,79.205182,5570.853552,61.0,-5.0,185.0,-17.0,72951.0
221591,30.863502,79.237513,5650.127409,61.0,110.0,185.0,331.0,73066.0
...,...,...,...,...,...,...,...,...
2499899,30.687314,79.452305,3569.479627,696.0,883.0,2088.0,2651.0,833299.0
2499942,30.687329,79.456318,3488.653082,696.0,898.0,2088.0,2694.0,833314.0
2500244,30.687341,79.484221,2769.532633,696.0,998.0,2088.0,2996.0,833414.0


time: 1.27 s (started: 2021-07-22 14:19:33 +00:00)


In [ ]:
photon_df=photon_df.drop(['latitude_index'],axis=1)
photon_df=photon_df.drop(['longitude_index'],axis=1)
photon_df=photon_df.drop(['latitude_index2'],axis=1)
photon_df=photon_df.drop(['longitude_index2'],axis=1)
photon_df.reset_index(drop=True, inplace=True)

time: 18.8 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
photon_df["container_box_id"] = photon_df["container_box_id"].astype(int)

time: 6.39 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
photon_df

,latitude,longitude,photon height,container_box_id
0,30.863485,79.179274,5934.822300,72857
1,30.863491,79.188049,5329.479165,72889
2,30.863479,79.204328,5572.857538,72948
3,30.863501,79.205182,5570.853552,72951
4,30.863502,79.237513,5650.127409,73066
...,...,...,...,...
80957,30.687314,79.452305,3569.479627,833299
80958,30.687329,79.456318,3488.653082,833314
80959,30.687341,79.484221,2769.532633,833414
80960,30.687324,79.484659,2762.454950,833416


time: 24.5 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
merged_df = photon_df.merge(dem_df, left_on='container_box_id', right_index = True)

time: 157 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
merged_df

,latitude_x,longitude_x,photon height,container_box_id,latitude_y,longitude_y,dem height
0,30.863485,79.179274,5934.822300,72857,30.863773,79.511528,4886.112793
1,30.863491,79.188049,5329.479165,72889,30.863766,79.520417,5079.104980
2,30.863479,79.204328,5572.857538,72948,30.863752,79.536806,5741.090332
3,30.863501,79.205182,5570.853552,72951,30.863751,79.537639,5756.089844
4,30.863502,79.237513,5650.127409,73066,30.863724,79.237361,5649.315918
...,...,...,...,...,...,...,...
80956,30.687326,79.452247,3571.340833,833299,30.687156,79.452083,3566.280518
80957,30.687314,79.452305,3569.479627,833299,30.687156,79.452083,3566.280518
80958,30.687329,79.456318,3488.653082,833314,30.687153,79.456250,3493.317627
80959,30.687341,79.484221,2769.532633,833414,30.687129,79.484028,2769.565918


time: 37.6 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
merged_df.drop(columns = ['container_box_id'], inplace = True)
X = merged_df.drop('photon height',axis=1)
y = merged_df['photon height']

time: 14.8 ms (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
from sklearn.model_selection import train_test_split

time: 1 s (started: 2021-07-22 14:19:34 +00:00)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

time: 19.2 ms (started: 2021-07-22 14:19:35 +00:00)


In [ ]:
import numpy as np
y_train = np.array(y_train)
y_test = np.array(y_test)

time: 4.51 ms (started: 2021-07-22 14:19:35 +00:00)


In [ ]:
y_train = y_train.reshape(len(y_train), 1)
y_test = y_test.reshape(len(y_test), 1)

time: 3.52 ms (started: 2021-07-22 14:19:35 +00:00)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_train= scaler_x.fit_transform(X_train)
X_test = scaler_x.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

time: 25.3 ms (started: 2021-07-22 14:19:35 +00:00)


In [ ]:
X_train.shape

(56673, 5)

time: 3.64 ms (started: 2021-07-22 14:19:35 +00:00)


In [ ]:
X_test.shape

(24289, 5)

time: 5.01 ms (started: 2021-07-22 14:19:36 +00:00)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam

time: 1.83 s (started: 2021-07-22 14:19:36 +00:00)


In [ ]:
model = Sequential()

model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam',loss='mse')

time: 274 ms (started: 2021-07-22 14:19:37 +00:00)


In [ ]:
model.fit(x=X_train,y=y_train,
          validation_data=(X_test,y_test),
          batch_size=128,epochs=200)

Epoch 1/200
443/443 [==============================] - 3s 5ms/step - loss: 0.0041 - val_loss: 0.0011
Epoch 2/200
443/443 [==============================] - 2s 4ms/step - loss: 9.5837e-04 - val_loss: 8.2304e-04
Epoch 3/200
443/443 [==============================] - 2s 4ms/step - loss: 8.7866e-04 - val_loss: 9.3667e-04
Epoch 4/200
443/443 [==============================] - 2s 4ms/step - loss: 7.9991e-04 - val_loss: 8.0675e-04
Epoch 5/200
443/443 [==============================] - 2s 4ms/step - loss: 7.8501e-04 - val_loss: 6.8793e-04
Epoch 6/200
443/443 [==============================] - 2s 4ms/step - loss: 7.7013e-04 - val_loss: 6.3332e-04
Epoch 7/200
443/443 [==============================] - 2s 4ms/step - loss: 7.3820e-04 - val_loss: 6.8908e-04
Epoch 8/200
443/443 [==============================] - 2s 4ms/step - loss: 7.2264e-04 - val_loss: 7.7189e-04
Epoch 9/200
443/443 [==============================] - 2s 4ms/step - loss: 7.2501e-04 - val_loss: 6.7173e-04
Epoch 10/200
443/443 [=====

In [ ]:
losses = pd.DataFrame(model.history.history)
losses.plot()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    model, show_shapes= True, show_dtype=False,
    show_layer_names=True, rankdir='LR', expand_nested=True, dpi=96

)

In [ ]:
model.save('ANN_Rahul.h5')

In [ ]:
finalarray=np.zeros([height*3*width*3], dtype = int)
df= pd.DataFrame(columns=['zero'],data=finalarray)
df

In [ ]:
def latitude_final(n):
    return origin_y + (n/(width*3) * pixel_res_y/3.0)
def longitude_final(n):
    return origin_x + (n%(width*3) * pixel_res_x/3.0)

In [ ]:
df=df.drop(columns=['index1','zero'],axis=1)
df['latitude_index']= df.latitude.apply(get_latitude_index)
df['longitude_index']= df.longitude.apply(get_longitude_index)
df['container_box_id']=(df['latitude_index'])*(width)+df['longitude_index']
df=df.drop(['latitude_index'],axis=1)
df=df.drop(['longitude_index'],axis=1)
df["container_box_id"] = df["container_box_id"].astype(int)

In [ ]:
final_merged_df = df.merge(dem_df, left_on='container_box_id', right_index = True)
final_merged_df.drop(columns = ['container_box_id'], inplace = True)

In [ ]:
final_merged_df

In [ ]:
final_merged_df.sort_index()

In [ ]:
X = final_merged_df
X = scaler_x.transform(X)

In [ ]:
predictions = model.predict(X)
final_merged_df['predictions']=predictions
final_merged_df

In [ ]:
final_merged_df['height']=scaler_y.inverse_transform(final_merged_df['predictions'].values.reshape(-1, 1))
final_merged_df

In [ ]:
final_merged_df=final_merged_df.sort_index()

In [ ]:
final_merged_df=final_merged_df.drop(columns=['latitude_x','longitude_x','latitude_y','longitude_y','predictions','dem height'])
final_merged_df

In [ ]:
finalarray = final_merged_df.to_numpy()
finalarray=finalarray.reshape(height*3 ,width*3)
finalarray

In [ ]:
dst_filename = 'badrinath_ann.tiff'
x_pixels = width*3  # number of pixels in x
y_pixels = height*3  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(finalarray)

In [ ]:
gtlist = list(gt)
gtlist[1]=gtlist[1]/3
gtlist[5]=gtlist[5]/3
gt=tuple(gtlist)

In [ ]:
geotrans = gt  #get GeoTranform from existed 'data0'
proj=ds.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [ ]:
from google.colab import files
files.download('badrinath_ann.tiff')